<a href="https://colab.research.google.com/github/Farouk96/Predict_tags_question/blob/main/Questions__tags_mod%C3%A9lisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from matplotlib import pyplot
import pickle

import time

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#One Vs rest
from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import Perceptron,LogisticRegression,LogisticRegressionCV,SGDClassifier,PassiveAggressiveClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.svm import LinearSVC 


#Multioutput
from sklearn.multioutput import MultiOutputClassifier

from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier

from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier



#Score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score, make_scorer

from sklearn.model_selection import GridSearchCV

from scipy.sparse import hstack

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/ P5_HADJ NACER_Farouk/Questions_Tags_Exploration.csv', sep=',')

# Data preparation


*   Binariser les tags
*   Utiliser un TFIDF pour body et Title Les paramètres du TFIDF sont très importants pour la performance de nos tags puisque nous ne voulons pas qu'il supprime des mots comme c# ou .net. Pour cela, nous devons utiliser le motif suivant : token_pattern=r"(?u)\S\S+"


In [ ]:
X1 = data['Body']
X2 = data['Title']
y = data['Tags']

In [ ]:
y

0                               ['<java']
1                         ['<typescript']
2                                  ['<c']
3                                ['<git']
4        ['<json', '<reactjs', '<django']
                       ...               
12225        ['<c#', '<string', '<loops']
12226          ['<python', '<matplotlib']
12227              ['<python', '<django']
12228         ['<scala', '<apache-spark']
12229                         ['<python']
Name: Tags, Length: 12230, dtype: object

In [ ]:
y =  [txt.replace("['<", "").replace("']", "").replace("'<","'").split("', '") for txt in y]

In [ ]:
y

[['java'],
 ['typescript'],
 ['c'],
 ['git'],
 ['json', 'reactjs', 'django'],
 ['c#'],
 ['react-native'],
 ['javascript', 'html', 'node.js', 'express'],
 ['python'],
 ['google-sheets'],
 ['php', 'mysql'],
 ['css'],
 ['python'],
 ['postgresql', 'macos'],
 ['vue.js'],
 ['python', 'python-3.x', 'pandas', 'dataframe'],
 ['java'],
 ['react-native'],
 ['flutter', 'dart'],
 ['javascript'],
 ['php', 'ajax'],
 ['java', 'spring', 'spring-boot'],
 ['android'],
 ['css', 'reactjs'],
 ['java', 'json', 'selenium'],
 ['html', 'asp.net-core'],
 ['git'],
 ['node.js', 'react-native', 'express'],
 ['linux'],
 ['node.js'],
 ['mysql'],
 ['c'],
 ['javascript', 'html', 'jquery', 'css', 'bootstrap-4'],
 ['scala'],
 ['c++', 'function'],
 ['mysql'],
 ['c#'],
 ['python'],
 ['postgresql'],
 ['android', 'android-studio'],
 ['python'],
 ['amazon-web-services'],
 ['powershell'],
 ['c++'],
 ['python', 'discord', 'discord.py'],
 ['html', 'css'],
 ['python', 'discord.py'],
 ['css', 'angular', 'typescript'],
 ['ios', 'sw

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [ ]:
multilabel_binarizer.classes_

array(['.net', '.net-core', 'ajax', 'algorithm', 'amazon-web-services',
       'android', 'android-studio', 'angular', 'apache-spark', 'api',
       'arrays', 'asp.net-core', 'azure', 'azure-devops', 'bash',
       'bootstrap-4', 'c', 'c#', 'c++', 'css', 'csv', 'dart', 'database',
       'dataframe', 'dictionary', 'discord', 'discord.py', 'django',
       'docker', 'dplyr', 'elasticsearch', 'excel', 'express', 'firebase',
       'flutter', 'for-loop', 'function', 'ggplot2', 'git',
       'google-cloud-firestore', 'google-sheets', 'html', 'if-statement',
       'ios', 'java', 'javascript', 'jquery', 'json', 'keras', 'kotlin',
       'kubernetes', 'laravel', 'linux', 'list', 'loops',
       'machine-learning', 'macos', 'matplotlib', 'mongodb', 'mongoose',
       'mysql', 'node.js', 'numpy', 'opencv', 'oracle', 'pandas', 'php',
       'postgresql', 'powershell', 'pyspark', 'python', 'python-3.x', 'r',
       'react-hooks', 'react-native', 'reactjs', 'regex', 'ruby',
       'ruby-on-rails'

In [ ]:
with open('Multi_label','wb') as S:
  pickle.dump(multilabel_binarizer,S)

In [ ]:
vectorizer_X1 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=5000)

vectorizer_X2 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=5000)

In [ ]:
X1_tfidf = vectorizer_X1.fit_transform(X1)
X2_tfidf = vectorizer_X2.fit_transform(X2)

In [ ]:
with open('Vectorizer question1','wb') as S:
  pickle.dump(vectorizer_X1,S)
with open('Vectorizer question2','wb') as T:
  pickle.dump(vectorizer_X2,T)  

In [ ]:
X_tfidf = hstack([X1_tfidf,X2_tfidf])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size = 0.2, random_state = 0) # Do 80/20 split
y_train.shape

(9784, 100)

#Modélisation supérvisé

In [ ]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("F1-score: {}".format(f1_score(y_test, y_pred,average='micro')))
    print("---")  

##One vs Rest
Pour évaluer nos modèles, j'utiliserai **le score de Jacard** car il est le mieux adapté à la classification multi-étiquettes.

In [ ]:
perceptron=Perceptron()
lr=LogisticRegression()
sgd=SGDClassifier()
svc=LinearSVC() 


for classifier in [sgd, lr, svc, perceptron]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  SGDClassifier
Jacard score: 49.57413464159172
F1-score: 0.6325010950503722
---
Clf:  LogisticRegression
Jacard score: 31.315072226764784
F1-score: 0.4627898926964348
---
Clf:  LinearSVC
Jacard score: 48.31084764240938
F1-score: 0.618598578267808
---
Clf:  Perceptron
Jacard score: 46.80080208698361
F1-score: 0.5926118626430801
---


##Multi output

In [ ]:
DT=DecisionTreeClassifier()
ETC=ExtraTreeClassifier()
KNC=KNeighborsClassifier()
RF=RandomForestClassifier()

In [ ]:
for classifier in [DT,ETC,KNC,RF]:
    clf = MultiOutputClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  DecisionTreeClassifier
Jacard score: 45.20538877856948
F1-score: 0.5784732100521575
---
Clf:  ExtraTreeClassifier
Jacard score: 28.404927383872597
F1-score: 0.39457175777749326
---
Clf:  KNeighborsClassifier
Jacard score: 17.326928318342873
F1-score: 0.30331753554502366
---
Clf:  RandomForestClassifier
Jacard score: 32.605614608885254
F1-score: 0.46186366038400567
---


##GridSearch CV on the best classifier

In [ ]:
param_grid = {'estimator__alpha':np.logspace(-10, 0,10),
              'estimator__penalty':["l1","l2","elasticnet"]
              } 

In [ ]:
sgd = OneVsRestClassifier(SGDClassifier())
CV_sgd = GridSearchCV(estimator=sgd, param_grid=param_grid, cv= 5,verbose=10, scoring=make_scorer(avg_jacard,greater_is_better=True))
CV_sgd.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5; 1/30] START estimator__alpha=1e-10, estimator__penalty=l1..............
[CV 1/5; 1/30] END estimator__alpha=1e-10, estimator__penalty=l1;, score=45.732 total time=   9.0s
[CV 2/5; 1/30] START estimator__alpha=1e-10, estimator__penalty=l1..............
[CV 2/5; 1/30] END estimator__alpha=1e-10, estimator__penalty=l1;, score=46.791 total time=   8.2s
[CV 3/5; 1/30] START estimator__alpha=1e-10, estimator__penalty=l1..............
[CV 3/5; 1/30] END estimator__alpha=1e-10, estimator__penalty=l1;, score=45.996 total time=   8.2s
[CV 4/5; 1/30] START estimator__alpha=1e-10, estimator__penalty=l1..............
[CV 4/5; 1/30] END estimator__alpha=1e-10, estimator__penalty=l1;, score=47.682 total time=   8.3s
[CV 5/5; 1/30] START estimator__alpha=1e-10, estimator__penalty=l1..............
[CV 5/5; 1/30] END estimator__alpha=1e-10, estimator__penalty=l1;, score=45.848 total time=   8.4s
[CV 1/5; 2/30] START estimator__alpha=

GridSearchCV(cv=5, estimator=OneVsRestClassifier(estimator=SGDClassifier()),
             param_grid={'estimator__alpha': array([1.00000000e-10, 1.29154967e-09, 1.66810054e-08, 2.15443469e-07,
       2.78255940e-06, 3.59381366e-05, 4.64158883e-04, 5.99484250e-03,
       7.74263683e-02, 1.00000000e+00]),
                         'estimator__penalty': ['l1', 'l2', 'elasticnet']},
             scoring=make_scorer(avg_jacard), verbose=10)

In [ ]:
CV_sgd.best_params_

{'estimator__alpha': 3.5938136638046256e-05, 'estimator__penalty': 'l1'}

In [ ]:
CV_sgd.best_estimator_

OneVsRestClassifier(estimator=SGDClassifier(alpha=3.5938136638046256e-05,
                                            penalty='l1'))

In [ ]:
best_model = CV_sgd.best_estimator_
y_pred = best_model.predict(X_test)

print_score(y_pred, best_model)

Clf:  OneVsRestClassifier
Jacard score: 51.129054238212056
F1-score: 0.6375168690958165
---


In [ ]:
with open('Best tager questions','wb') as f:
  pickle.dump(best_model,f)

In [ ]:
best_model.n_features_in_

10000